In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# 개요
[참고한 사이트](https://www.kaggle.com/code/wintersadh/spaceship-titanic-plotly-eda-lgbm-ee4adf/edit) SpaceTitanic Kaggle Competition 링크 글을 참고하여 코드를 따라쓰며 필자가 해석한 부분을 설명하면서 머신러닝에 대해 이해하기 위한 목적으로 작성하게 되었습니다.

# Step 1. 라이브러리 및 데이터 불러오기

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go

- 라이브러리 버전 확인

In [3]:
print(np.__version__)
print(pd.__version__)
print(sns.__version__)

1.20.3
1.3.5
0.11.2


- 데이터를 불러올때 저장경로 앞에 r을 쓰는 이유는 \n, \r 같은 이스케이프 문자(Escape sequence)를 적용하지 않고 인용 부호 안의 문자를 그대로 출력하는 Raw문자열을 만들기 위함입니다.
[참고](https://kkoseul.tistory.com/50)

In [4]:
train = pd.read_csv(r'../input/spaceship-titanic/train.csv')
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
test = pd.read_csv(r'../input/spaceship-titanic/test.csv')
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [6]:
sub = pd.read_csv(r'../input/spaceship-titanic/sample_submission.csv')
sub.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


- f-string 포매팅을 이용한 차원 확인

In [7]:
print(f'train set have {train.shape[0]} rows and {train.shape[1]} columns.')
print(f'test set have {test.shape[0]} rows and {test.shape[1]} columns.')
print(f'sample_comission set have {sub.shape[0]} rows and {sub.shape[1]} columns.')

train set have 8693 rows and 14 columns.
test set have 4277 rows and 13 columns.
sample_comission set have 4277 rows and 2 columns.


- 데이터 타입 확인

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


- 컬럼별 레이블의 개수 확인
    + data_Frame.nunique() = 데이터셋에 존재하는 컬럼별 레이블의 개수를 확인하기 위해 사용

In [9]:
train.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

- 필요없는 칼럼 정리
    + drop(inplce = True) : 기존 프레임에 변경된 설정으로 덮어쓰겠다는 의미, 그렇기 때문에 다시 데이터 프레임을 가져왔을때 출력되지 않음. [참고링크](https://www.dinolabs.ai/70)

In [10]:
train.drop(['PassengerId','Name', 'Cabin'], axis = 1, inplace = True)
test.drop(['PassengerId','Name', 'Cabin'], axis = 1, inplace = True)

- 훈련 테스트 결측값 여부 확인
    + data_Frame.isnull() method는 관측치가 결측이면 True, 결측이 아니면 False의 boolean 값을 반환
    + data_Frame.isnull().sum(): 결측값의 개수를 반환
    + [참고링크](https://rfriend.tistory.com/260)

In [11]:
train.isnull().sum()

HomePlanet      201
CryoSleep       217
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

- 주요 통계량 요약(summary) 전치행렬
    + data_frame.describe().T : describe()로 요약한 통계량의 행렬을 바꿈(transpose) 
    + [참고링크](https://www.geeksforgeeks.org/pandas-dataframe-t-function-in-python/)

In [12]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,8514.0,28.827930,14.489021,0.0,19.0,27.0,38.0,79.0
RoomService,8512.0,224.687617,666.717663,0.0,0.0,0.0,47.0,14327.0
FoodCourt,8510.0,458.077203,1611.489240,0.0,0.0,0.0,76.0,29813.0
ShoppingMall,8485.0,173.729169,604.696458,0.0,0.0,0.0,27.0,23492.0
Spa,8510.0,311.138778,1136.705535,0.0,0.0,0.0,59.0,22408.0
VRDeck,8505.0,304.854791,1145.717189,0.0,0.0,0.0,46.0,24133.0


- 결측값을 missing 으로 변환

In [13]:
train['HomePlanet'].replace(np.nan, 'missing', inplace = True)
train['CryoSleep'].replace(np.nan, 'missing', inplace = True)
train['Destination'].replace(np.nan, 'missing', inplace = True)
train['VIP'].replace(np.nan, 'missing', inplace = True)

test['HomePlanet'].replace(np.nan, 'missing', inplace = True)
test['CryoSleep'].replace(np.nan, 'missing', inplace = True)
test['Destination'].replace(np.nan, 'missing', inplace = True)
test['VIP'].replace(np.nan, 'missing', inplace = True)

- 최빈값 확인
    + 가장 빈번하게 관찰/측정 되는 값

In [14]:
train['Age'].mode()

0    24.0
dtype: float64

In [15]:
train['RoomService'].fillna(train['RoomService'].mean(), inplace = True)
train['FoodCourt'].fillna(train['FoodCourt'].mean(), inplace = True)
train['ShoppingMall'].fillna(train['ShoppingMall'].mean(), inplace = True)
train['Spa'].fillna(train['Spa'].mean(), inplace = True)
train['VRDeck'].fillna(train['VRDeck'].mean(), inplace = True)
train['Age'].replace(np.nan, 24.0 , inplace = True)

- 결측값 missing으로 변환됬는지 확인

In [16]:
train.isnull().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

# Step 2. 탐색적 자료 분석 (EDA)
- 데이터 시각화
- 산점도, 막대 그래프 등
- 그래프 해석해서 설명을 달아야 함
- 약간의 데이터 전처리

# Step 3. 데이터 전처리
- Feature Engineering
- ML 모형을 돌리기 위해 표준화 등 / 원핫-인코딩
- 파생변수 (도출 변수)
    + 왜 이 변수를 만들었는지에 대한 설명 필요

# Step 4. 머신러닝 모형 개발
- 모형에 대한 설명 필요
- 모형을 1~2개 사용
- 교차 검증
- 하이퍼 파라미터 튜닝

# Step 5. 모형 평가
- 훈련데이터 쪼갠다. 훈련데이터 + 검증데이터 분리
- 정확도 비교
- 혼동행렬 (Confusion Matrix) 설명

# Step 6. 제출
- 제출 양식은 샘플 만들어드림

# 참고
- 다른 사람의 code 설명을 쭉 따라침.
- 노트북 표절 방지 위해, 여러분들이 참조한 코드는 반드시 링크 걸어둘 것
- 저자 이름, 글 제목, 링크 주소

# 마감일
- 4월 12일 17시 40분
- 제출 형태
    + Leaderboard 랭킹 사진 캡쳐
    + 고용노동부 보고 양식 (다음주에 확인해서 알려드림)